### Description
- The objective of this agent is to provide it with a task, devide the task in steps.
- Have several agents doing each task in paralell
### Solution
- Provide a task to the LLM, ask it to provide the steps to do the task, the answer should be in json
- Parse the json dividing the several tasks.
- In paralell call the LLM providing each task, asking it to provide the solution
- Have a Validator in the end that receives each task description, solution and validate that they will work well together

### Setting up the environment 
In the future i'll be using the library utils/myopenai.py
that does the bellow and has a function that gets openai instances

In [18]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os

load_dotenv(override=True)
openai_api_key = os.getenv('GOOGLE_API_KEY')
openai_url = os.getenv('GEMINI_BASE_URL')
gemini_model = os.getenv('GEMINI_MODEL')

def get_openai_client():
    return OpenAI(base_url=openai_url, api_key=openai_api_key)

def send_message(client, message,role="user", model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=[
            {"role": role, "content": message}
        ]
    )

def send_messages(client, messages, model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

In [16]:
question_ai = get_openai_client()

response = send_message(question_ai, 
"I want to create a really simple Android app, This app will have 1 activity.\
The objective of this activity is to enable user to create a user or login to a existing user. \
Create tasks on how to perform this objective. I will use the tasks description to provide to another AI agent to perform the task\
So each description should also have the context of the current task, and what needs to be done \
please provide a response in json with a step number and task description, don't use markdown or any other output but json")

result = response.choices[0].message.content
print(result)




```json
[
  {
    "step": 1,
    "taskDescription": "Create a new Android project with an Empty Activity. Name the project 'UserAuthApp' and set the package name to 'com.example.userauthapp'. Choose Kotlin as the language."
  },
  {
    "step": 2,
    "taskDescription": "Design the layout for the main activity (activity_main.xml). Include two Button widgets: one labeled 'Create User' and another labeled 'Login'.  Also, include two EditText widgets for user input: one for 'Username' and another for 'Password'.  Add TextView widgets for labels to guide the user (e.g., 'Username:', 'Password:').  Arrange these elements using appropriate layout managers (e.g., LinearLayout, ConstraintLayout) for a clean and user-friendly interface."
  },
  {
    "step": 3,
    "taskDescription": "In MainActivity.kt, find the 'Create User' button and the 'Login' button by their IDs. Set an OnClickListener for each button.  Initially, these listeners will be placeholders and won't perform any action."
  },
 

In [19]:
results_list = json.loads(result)

for result in results_list:
    print(result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)